In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

In [2]:
!dir

1429_1.csv  sample_data


In [3]:
data1 = pd.read_csv('1429_1.csv')
data1.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


Removing missing review text from the data

In [4]:
text1 = data1['reviews.text'].dropna(axis = 0, how ='any') 
type(text1)

pandas.core.series.Series

In [5]:
len(text1)

2355

In [6]:
len(data1['reviews.text'] )

2356

In [14]:
!python -m spacy download  'en_core_web_sm'

     |████████████████████████████████| 12.0 MB 15.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [66]:
nlp = spacy.load('en_core_web_sm')
keywords = []
mm = text1
for sen in mm:
    docs = nlp(sen)
    keyw = ''
    for token in docs:
        if(token.pos_ == 'ADJ'):
            x = ''
            for m in token.children:
                if(m.pos_ != 'ADV'):
                    continue
                x+= m.text + ' '
            keyw = x + token.text
    keywords.append(keyw)
    
    print(f'{sen} and their keywords: {keyw}')
    print()
mm = pd.DataFrame(mm)
mm['KeyWords'] = keywords

Streaming output truncated to the last 5000 lines.
I always buy these kindles for corporate giveaways. They're reasonably priced and are great quality. and their keywords: great

Compact, light, and extremely easy on the eyes when reading. and their keywords: extremely easy

This is the 3rd Paperwhite I've purchased... for me and as gifts. Absolutely love them! and their keywords: 3rd

this kindle is absolutely awesome! it has a great reading light on there and is easy to use. i would highly recommend it to anyone who is looking for an e-book reader. and their keywords: easy

Paper white does not allow you do use books from the library as you cannot load apps. Only books from Amazon are available to read. and their keywords: available

I absolutely love my Kindle Paperwhite. No glare no matter where I am reading. It is lightweight and the right size to take along in my purse. I had never had a kindle before and I am amazed with it! and their keywords: amazed

Excellent in all light con

In [67]:
print(mm.shape)
print(mm.head(10))

(20382, 2)
                                        reviews.text          KeyWords
0  This product so far has not disappointed. My c...                  
1  great for beginner or experienced person. Boug...       experienced
2  Inexpensive tablet for him to use and learn on...       Inexpensive
3  I've had my Fire HD 8 two weeks now and I love...                ur
4  I bought this for my grand daughter when she c...              most
5  This amazon fire 8 inch tablet is the perfect ...              good
6  Great for e-reading on the go, nice and light ...  definitely worth
7  I gave this as a Christmas gift to my inlaws, ...         fantastic
8  Great as a device to read books. I like that i...            Kindle
9  I love ordering books and reading them with th...                  


In [68]:
len(keywords)

20382

### Method - 2: Using BERT encodding and matching keywords with review context using cosine similarity

In [7]:
!pip install keybert

     |████████████████████████████████| 78 kB 6.0 MB/s 
     |████████████████████████████████| 214 kB 35.7 MB/s 
     |████████████████████████████████| 51 kB 6.2 MB/s 
     |████████████████████████████████| 3.4 MB 38.9 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
     |████████████████████████████████| 1.2 MB 25.6 MB/s 
     |████████████████████████████████| 61 kB 411 kB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 895 kB 26.4 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=612a144ef898f8044ba7115bdeb86ae41f1097a1c2f2ccdaf1590548d369fff5
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=20f43778518ef5284ea1976b36eea6893ba9edea1f591030460fffc99f3a37d6
  Stored in directory: /root/.cache/pip/whee

In [8]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
keyw = KeyBERT(model=sentence_model)
text2 = text1
mm = text2
kws = []
keywordsUsingBERT = []
for sen in mm:
    kw = keyw.extract_keywords(sen,keyphrase_ngram_range=(1, 2), stop_words=None,use_maxsum=True, nr_candidates=20, top_n=1)
    # k = ''
    # max=0
    # for x in kw:
    #   #print(x[0],x[1])
    #   if(x[1]>max):
    #     max=x[1]
    #     k=x[0]
    #print(k)
    keywordsUsingBERT.append(kw[0][0])
     


In [38]:
text2 = pd.DataFrame(mm)
text2['KeyWords'] = keywordsUsingBERT 

In [39]:
len(keywordsUsingBERT)

2355

In [41]:
text2

,reviews.text,KeyWords
0,This product so far has not disappointed. My c...,content they
1,great for beginner or experienced person. Boug...,it
2,Inexpensive tablet for him to use and learn on...,learn
3,I've had my Fire HD 8 two weeks now and I love...,prime content
4,I bought this for my grand daughter when she c...,sites and
...,...,...
2350,"This is an awesome tablet, fast, great HD disp...",hd
2351,Great size. Works with all apps. No issues yet...,would buy
2352,"bought this for my son(12), and he loves it. C...",the holidays
2353,This Amazon Fire is perfect for what I need - ...,price was
